In [1]:
reasoning_system_prompt = open("reasoning_system_prompt.txt", "r").read()

reasoning_user_prompt = open("reasoning_user_prompt.txt", "r").read()

answering_system_prompt = open("answering_system_prompt.txt", "r").read()

In [2]:
question = "I have pain in my chest and shortness of breath since yesterday."
demographics = "46 years old male, smoker, high blood pressure"

In [3]:
# set up vector retriever with: t0-1 serve-vector-store
import requests

def query_vector_store(query, k=10, with_score=True):
    url = "http://127.0.0.1:8000/query"
    params = {
        "query": query,
        "k": k,
        "with_score": with_score
    }
    response = requests.get(url, params=params, headers={"accept": "application/json"})
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Request failed with status code {response.status_code}")
        return None
context = query_vector_store(question, k=5, with_score=True)

In [4]:
sources = [context['response'][x][0]['metadata']['source'] for x in range(len(context['response']))]

In [5]:
reasoning_user_prompt = reasoning_user_prompt.format(
    context=context,
    question=question,
    demographics=demographics,
    sources=sources
)

In [ ]:
from ollama import AsyncClient

thinking_init = "<|im_start|>think\n"

reasoning_messages = [
    {"role": "system", "content": reasoning_system_prompt},
    {"role": "user", "content": reasoning_user_prompt + thinking_init}
]


async def reasoning(messages):
    reasoning_trace = []
    client = AsyncClient()   # create the client
    async for part in await client.chat(
            model="hf.co/rchan26/t0-k5-32B-Q4_K_M-GGUF",
            messages=messages,
            stream=True,
            options={"stop": ["<|im_start|>"]}
        ):
        reasoning_trace.append(part['message']['content'])
    return ''.join(reasoning_trace)


async def answering(messages):
    client = AsyncClient()   # create the client
    async for part in await client.chat(
            model="gemma3:1b",
            messages=messages,
            stream=True,
            options={"stop": ["<|im_end|>"]}
        ):
        print(part['message']['content'], end='', flush=True)

reasoning_trace = await reasoning(reasoning_messages)

In [ ]:
answering_messages = [
    {"role": "system", "content": answering_system_prompt},
    {"role": "user", "content": reasoning_trace}
]

await answering(answering_messages)